Applicazione della regressione multivariata per cercare di predire l'evoluzione dei tempi sul giro, date le simulazione eseguite in FP2

In [ ]:
import fastf1 as ff1
import pandas as pd
from fastf1.core import Laps
from fastf1 import plotting
from fastf1 import utils
from matplotlib import pyplot as plt
from timple.timedelta import strftimedelta
import numpy as np
from matplotlib.pyplot import figure
from matplotlib.collections import LineCollection
from matplotlib import cm


fp = ff1.get_session(2023, 'Singapore', 'FP2')
#fp_laps = fp.load_laps(with_telemetry=True)
fp.load()

Carico i dati della RedBull

In [72]:
drivers = pd.unique(fp.laps['Driver'])
driver_laps_static = fp.laps.pick_drivers(drivers)

In [ ]:
driver_laps = driver_laps_static.loc[((driver_laps_static['Driver'] == "VER")  & (driver_laps_static['IsAccurate']==True) & (driver_laps_static['Stint']==3)) |
                                     ((driver_laps_static['Driver'] == "PER")  & (driver_laps_static['IsAccurate']==True) & (driver_laps_static['Stint']==4))]
driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
driver_laps["Time(s)"] = driver_laps["Time"].dt.total_seconds()
driver_laps["Sector1Time(s)"] = driver_laps["Sector1Time"].dt.total_seconds()
driver_laps["Sector2Time(s)"] = driver_laps["Sector2Time"].dt.total_seconds()
driver_laps["Sector3Time(s)"] = driver_laps["Sector3Time"].dt.total_seconds()

In [74]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [75]:
driver_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,LapTime(s),Time(s),Sector1Time(s),Sector2Time(s),Sector3Time(s)
13,0 days 01:04:22.821000,VER,1,0 days 00:01:42.042000,14.0,3.0,NaT,NaT,0 days 00:00:29.018000,0 days 00:00:41.897000,...,NaN,False,,False,True,102.042,3862.821,29.018,41.897,31.127
14,0 days 01:06:01.042000,VER,1,0 days 00:01:38.221000,15.0,3.0,NaT,NaT,0 days 00:00:28.869000,0 days 00:00:41.542000,...,NaN,False,,False,True,98.221,3961.042,28.869,41.542,27.810
15,0 days 01:07:43.921000,VER,1,0 days 00:01:42.879000,16.0,3.0,NaT,NaT,0 days 00:00:29.257000,0 days 00:00:41.777000,...,NaN,False,,False,True,102.879,4063.921,29.257,41.777,31.845
16,0 days 01:09:22.251000,VER,1,0 days 00:01:38.330000,17.0,3.0,NaT,NaT,0 days 00:00:29.028000,0 days 00:00:41.597000,...,NaN,False,,False,True,98.330,4162.251,29.028,41.597,27.705
17,0 days 01:11:00.818000,VER,1,0 days 00:01:38.567000,18.0,3.0,NaT,NaT,0 days 00:00:29.003000,0 days 00:00:41.820000,...,NaN,False,,False,True,98.567,4260.818,29.003,41.820,27.744
18,0 days 01:12:39.397000,VER,1,0 days 00:01:38.579000,19.0,3.0,NaT,NaT,0 days 00:00:28.919000,0 days 00:00:41.868000,...,NaN,False,,False,True,98.579,4359.397,28.919,41.868,27.792
19,0 days 01:14:18.046000,VER,1,0 days 00:01:38.649000,20.0,3.0,NaT,NaT,0 days 00:00:28.978000,0 days 00:00:41.861000,...,NaN,False,,False,True,98.649,4458.046,28.978,41.861,27.810
20,0 days 01:15:57.211000,VER,1,0 days 00:01:39.165000,21.0,3.0,NaT,NaT,0 days 00:00:28.884000,0 days 00:00:42.557000,...,NaN,False,,False,True,99.165,4557.211,28.884,42.557,27.724
21,0 days 01:18:22.262000,VER,1,0 days 00:02:25.051000,22.0,3.0,NaT,NaT,0 days 00:00:38.072000,0 days 00:00:57.699000,...,NaN,False,,False,True,145.051,4702.262,38.072,57.699,49.280
114,0 days 01:09:26.416000,PER,11,0 days 00:01:38.078000,17.0,4.0,NaT,NaT,0 days 00:00:28.932000,0 days 00:00:41.497000,...,NaN,False,,False,True,98.078,4166.416,28.932,41.497,27.649


rimuovo le colonne che non sono utili per la regressione

In [76]:
colonne_da_tenere = ["LapNumber", "Compound", "TyreLife", "FreshTyre", "TrackStatus", "LapTime(s)"]
driver_laps_dropped = driver_laps[colonne_da_tenere]

In [77]:
driver_laps_dropped

,LapNumber,Compound,TyreLife,FreshTyre,TrackStatus,LapTime(s)
13,14.0,MEDIUM,9.0,False,1,102.042
14,15.0,MEDIUM,10.0,False,1,98.221
15,16.0,MEDIUM,11.0,False,1,102.879
16,17.0,MEDIUM,12.0,False,1,98.330
17,18.0,MEDIUM,13.0,False,1,98.567
18,19.0,MEDIUM,14.0,False,1,98.579
19,20.0,MEDIUM,15.0,False,1,98.649
20,21.0,MEDIUM,16.0,False,1,99.165
21,22.0,MEDIUM,17.0,False,1,145.051
114,17.0,MEDIUM,11.0,False,1,98.078


converto le features nominali in numeriche

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
driver_laps_dropped["Compound"] = le.fit_transform(driver_laps_dropped["Compound"])
driver_laps_dropped["FreshTyre"] = driver_laps_dropped["FreshTyre"].astype(int)

In [79]:
driver_laps_dropped

,LapNumber,Compound,TyreLife,FreshTyre,TrackStatus,LapTime(s)
13,14.0,0,9.0,0,1,102.042
14,15.0,0,10.0,0,1,98.221
15,16.0,0,11.0,0,1,102.879
16,17.0,0,12.0,0,1,98.330
17,18.0,0,13.0,0,1,98.567
18,19.0,0,14.0,0,1,98.579
19,20.0,0,15.0,0,1,98.649
20,21.0,0,16.0,0,1,99.165
21,22.0,0,17.0,0,1,145.051
114,17.0,0,11.0,0,1,98.078


In [80]:
features = ["LapNumber", "Compound", "TyreLife", "FreshTyre", "TrackStatus"]
target = "LapTime(s)"

X = driver_laps_dropped[features]
y = driver_laps_dropped[target]

# Suddividi il dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inizializza il modello di regressione lineare
model = LinearRegression()

# Addestra il modello sui dati di addestramento
model.fit(X_train, y_train)

# Fai previsioni sui dati di test
y_pred = model.predict(X_test)

# Calcola le metriche di valutazione del modello
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 75.04896175850473
R-squared (R2): -21.59564722518275


In [81]:
new_data = pd.DataFrame({
    "LapNumber" : list(range(14, 34)),
    "Compound" : [0]*20,
    "TyreLife" : list(range(1, 21)),
    "FreshTyre" : [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    "TrackStatus" : [1]*20
})

In [82]:
predictions = model.predict(new_data)

print("Previsioni di LapTime(s) per RedBull\nDal giro 13 al 33 su gomma MEDIUM:\n")
for prediction in predictions:
    print(prediction)
print("\nNB: Pochi dati di training")

Previsioni di LapTime(s) per RedBull
Dal giro 13 al 33 su gomma MEDIUM:

5.578630357142913
8.894762500000056
12.210894642857212
15.527026785714355
18.843158928571484
22.159291071428655
25.475423214285783
28.79155535714294
32.10768750000007
35.423819642857225
38.73995178571437
42.056083928571525
45.37221607142867
48.688348214285824
52.00448035714295
55.32061250000011
58.63674464285724
61.952876785714395
65.26900892857152
68.58514107142865

NB: Pochi dati di training


MLP

In [83]:
from sklearn.neural_network import MLPRegressor

In [84]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

In [85]:
pred = regr.predict(new_data)

print("Previsioni di LapTime(s) per RedBull\nDal giro 13 al 33 su gomma MEDIUM:\nModello: MLP")
for prediction in pred:
    print(prediction)

Previsioni di LapTime(s) per RedBull
Dal giro 13 al 33 su gomma MEDIUM:
Modello: MLP
51.060048698186115
57.07284013089172
63.113637256541985
69.1545858312612
75.19553440614898
81.23648298103676
87.27743155592452
93.3183801308123
99.35253676789952
105.38657800527953
111.42061924265954
117.45466048003955
123.48870171741954
129.52274295479955
135.55678419217955
141.59082542955954
147.62486666693957
153.6589079043196
159.69294914169956
165.72699037907958
